## Importing Libraries

In [ ]:
#! pip install pandas
#! pip install pathlib
#! pip install numpy
#! pip install matplotlib
#! pip install ipykernel

In [17]:
# Importing relevant libraries
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib
import matplotlib.pyplot as plt
import math

ModuleNotFoundError: No module named 'pandas'

## Defining Functions

In [5]:
def remove_first(x):
  x =  x[1:]
  x = float(x)
  return x

def remove_last(x):
  x = x[:(len(x)-1)]
  x = float(x)
  return x

def revalue(x):
  if x >180:
    x= -(360-x)
  return x

In [6]:
def process_data(filepath):
  
  b = pd.read_csv(filepath)

  b = b.iloc[3:]
  b = b.reset_index(drop=True)

  b.rename(columns={'Lapsed Time': 'Time'}, inplace=True)
  b.rename(columns={'Target Name': 'Target_Name'}, inplace=True)

  b['Time_Diff'] = b['Time'].diff()
  b['Time_Diff'] = b['Time_Diff'].apply(lambda x: max(0, x))

  b['X Euler Angle'] = b['X Euler Angle'].apply(remove_first)
  b['Z Euler Angle'] = b['Z Euler Angle'].apply(remove_last)

  b['X_A'] = b['X Euler Angle']
  b['Y_A'] = b['Y Euler Angle']
  b['Z_A'] = b['Z Euler Angle']

  b['X_A_Rev'] = b['X Euler Angle'].apply(revalue)
  b['Y_A_Rev'] = b['Y Euler Angle'].apply(revalue)
  b['Z_A_Rev'] = b['Z Euler Angle'].apply(revalue)

  b['X_A_Rev_Diff'] = abs(b['X_A_Rev'].diff())
  b['Y_A_Rev_Diff'] = abs(b['Y_A_Rev'].diff())
  b['Z_A_Rev_Diff'] = abs(b['Z_A_Rev'].diff())

  # Removing all rows that say Mission Complete
  label_to_remove = 'Mission complete'
  b = b[b['Target_Name'] != label_to_remove]

  b.fillna(0, inplace=True)

  b_df = b[['Target_Name', 'X', 'Z', 'X_A', 'X_A_Rev', 'X_A_Rev_Diff', 'Y_A', 'Y_A_Rev', 'Y_A_Rev_Diff', 'Z_A', 'Z_A_Rev', 'Z_A_Rev_Diff', 'Time', 'Time_Diff']].copy()
  
  return b_df 

In [5]:
def get_ct(data):
    
    grouped = data.groupby('Target_Name')
    results = pd.DataFrame(columns=['Target_Name', 'Total_Time'])

    # Iterate through each group
    for label, group_data in grouped:
        sum = group_data['Time_Diff'].sum()

        # Store in a df with this iteration's results
        iteration_df = pd.DataFrame({'Target_Name': [label], 'Total_Time': [sum]})

        # Add to total results df
        results = pd.concat([results, iteration_df], ignore_index=True)
    
    results = results.set_index('Target_Name', drop=True)
    return(results)

In [6]:
def get_ori_ct(data):
    
    grouped = data.groupby('Target_Name')
    results = pd.DataFrame(columns=['Target_Name', 'Orientation_Time'])

    # Iterate through each group
    for label, group_data in grouped:
        # Initialize a count variable
        count = 0

        # Iterate through the DataFrame
        for index, row in group_data.iterrows():
            if row['X'] == 0 and row['Z'] == -4.1:
                count += row['Time_Diff']

        # Store in a df with this iteration's results
        iteration_df = pd.DataFrame({'Target_Name': [label], 'Orientation_Time': [count]})

        # Add to total results df
        results = pd.concat([results, iteration_df], ignore_index=True)
    
    results = results.set_index('Target_Name', drop=True)
    return(results)


In [7]:
def get_nav_ct(data):
    
    grouped = data.groupby('Target_Name')
    results = pd.DataFrame(columns=['Target_Name', 'Navigation_Time'])

    # Iterate through each group
    for label, group_data in grouped:
        # Initialize a count variable
        count = 0

        # Iterate through the DataFrame
        for index, row in group_data.iterrows():
            if row['X'] == 0 and row['Z'] == -4.1:
                continue
            else:
                count += row['Time_Diff']

        # Store in a df with this iteration's results
        iteration_df = pd.DataFrame({'Target_Name': [label], 'Navigation_Time': [count]})

        # Add to total results df
        results = pd.concat([results, iteration_df], ignore_index=True)
    
    results = results.set_index('Target_Name', drop=True)
    return(results)

In [8]:
def get_dt(data):
  
  # Selecting columns
  df = data.drop_duplicates(subset = ['X','Z','Target_Name'],keep = ('first'))

  # Creating columns that we need
  df['X_Diff'] = df['X'].diff()
  df['Z_Diff'] = df['Z'].diff()
  df = df[['Target_Name', 'X_Diff', 'Z_Diff','Time_Diff']].copy()

  df['Distance'] = np.sqrt((df['X_Diff']**2) + (df['Z_Diff']**2))
  dt = df.groupby(['Target_Name'])['Distance'].sum()

  dt = pd.DataFrame(dt)

  return(dt)

In [9]:
def get_dwell(data):
    # Group the DataFrame by 'Label'
    grouped = data.groupby('Target_Name')

    results = pd.DataFrame(columns = ['Target_Name', 'Mean_Dwell'])

    # Iterate through each group
    for label, group_data in grouped:
        pos_grouped = group_data.groupby(['X', 'Z'])
        times = []

        for group_keys, group_data in pos_grouped:
            list = pd.DataFrame(group_data)
            first = list['Time'].iloc[0]
            last = list['Time'].iloc[-1]
            diff = last - first
            times.append(diff)
        
        times.pop(0)
        mean_dwell = np.mean(times)
        times = []

        # Store in a df with this iteration's results
        iteration_df = pd.DataFrame({'Target_Name': [label], 'Mean_Dwell': [mean_dwell]})

        # Add to total results df
        results = pd.concat([results, iteration_df], ignore_index=True)

    results = results.set_index('Target_Name', drop=True)
    return(results)

In [10]:
def get_teleport(data):
    grouped = data.groupby('Target_Name')
    results = pd.DataFrame(columns=['Target_Name', 'Teleportations'])

    # Iterate through each group
    for label, group_data in grouped:
    # Create an empty set to store unique positions
        unique_positions = set()

        # Iterate through the DataFrame
        for index, row in group_data.iterrows():
            x, y = row['X'], row['Z']
            
            # Add the (X, Y) tuple to the set
            unique_positions.add((x, y))

        # Calculate the number of unique positions
        num_unique_positions = len(unique_positions)

        # Store in a df with this iteration's results
        iteration_df = pd.DataFrame({'Target_Name': [label], 'Teleportations': [num_unique_positions]})

        # Add to total results df
        results = pd.concat([results, iteration_df], ignore_index=True)

    results = results.set_index('Target_Name', drop=True)
    return(results)


## Getting All Data

In [3]:
# Retrieving filepath to all data folders
#get_ipython().run_line_magic('store -r', 'fp_folder')
%store -r fp_folder

sub_folders = []
for i in os.listdir(fp_folder):
    if os.path.isdir(os.path.join(fp_folder, i)) and (i.startswith('BNC') or i.startswith('NAV')):
        sub_folders.append(i)

num_subjects = len(sub_folders)
sub_folders = sorted(sub_folders)
print(sub_folders)

['BNC01', 'BNC02', 'BNC03', 'BNC04', 'BNC05', 'BNC07', 'BNC08', 'BNC09', 'BNC11', 'BNC12', 'BNC23', 'BNC26', 'BNC27', 'BNC28', 'BNC29', 'BNC30', 'BNC31', 'BNC32', 'BNC33', 'BNC34', 'BNC35', 'BNC36', 'BNC37', 'BNC50', 'BNC51', 'BNC52']


In [14]:
# Iterating through participant folders and conducting analyses
for PID in sub_folders:
    
    # Filepath to each participant folder
    fp_PID = fp_folder + PID

    for i in range(1,4):
        fp_data = fp_PID +'/Saved_data_'+ PID + '_t' + str(i) + '.csv'

        data = process_data(fp_data)

        ct_results = get_ct(data)
        ori_ct_results = get_ori_ct(data)
        nav_ct_results = get_nav_ct(data)
        dt_results = get_dt(data)
        dwell_results = get_dwell(data)
        teleport_results = get_teleport(data)
        speed_results = dt_results['Distance'] / nav_ct_results['Navigation_Time']
        speed_results = speed_results.rename('Speed')

        final = pd.concat([ct_results, ori_ct_results, nav_ct_results, dt_results, speed_results, dwell_results, teleport_results], axis=1)

        # Define the desired order of indices
        new_order = ['Automobile shop', 'Police station ', 'Fire Station', 'Bank', 'Pawn Shop', 'Pizzeria', 'Quattroki Restaurant', 'High School']

        # Reorganize the DataFrame based on the new index order
        final = final.reindex(new_order)
        
        block = f"b{i}"
        final.to_csv(fp_PID + '/' + block + '_results.csv', index=True)

/var/folders/vd/hgkjsb1j7gj04s6dypg9lc0c0000gn/T/ipykernel_52417/4279315902.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['X_Diff'] = df['X'].diff()
/var/folders/vd/hgkjsb1j7gj04s6dypg9lc0c0000gn/T/ipykernel_52417/4279315902.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Z_Diff'] = df['Z'].diff()
/var/folders/vd/hgkjsb1j7gj04s6dypg9lc0c0000gn/T/ipykernel_52417/4279315902.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [16]:
# Define the colors for each target building
target_colors = {
    'Automobile shop': '#000000',
    'Police station': '#ff0010',
    'Fire Station': '#ff55c2',
    'Bank': '#9250fb',
    'Pawn Shop': '#00b9ff',
    'Pizzeria': '#034cb4',
    'Quattroki Restaurant': '#00c359',
    'High School': '#ff8a33'
}

for PID in sub_folders:
    
    # Filepath to each participant folder
    fp_PID = fp_folder + PID

    for i in range(1, 4):
        fp_data = fp_PID + '/Saved_data_' + PID + '_t' + str(i) + '.csv'

        # Assuming process_data is a function that processes and returns a DataFrame
        data = process_data(fp_data)

        plt.figure(figsize = (8,8))
        
        # Plot the X-Z movement for each target, using the specified colors
        for target_name, target_data in data.groupby('Target_Name'):
            color = target_colors.get(target_name, '#000000')  # Use black as default color if target not found
            plt.plot(target_data['X'], target_data['Z'], color=color, linewidth=3, label=target_name)
            plt.xlim(-80, 80)
            plt.ylim(-60, 80)
            plt.xticks(fontsize=20)
            plt.yticks(fontsize=20)

        # Define the block and image path
        block = f"b{i}"
        imagepath = Path(fp_PID + '/' + block + '_movement.png') 
        imagepath.parent.mkdir(parents=False, exist_ok=True)
        
        # Save the plot
        plt.savefig(imagepath)

        # Clear the figure for the next iteration
        plt.clf()

/var/folders/vd/hgkjsb1j7gj04s6dypg9lc0c0000gn/T/ipykernel_16081/4275418646.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize = (8,8))


<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

...

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>